# Connect to Snowflake

In [ ]:
# access data from snowflake
import pandas as pd
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *

connection_parameters = {
    "account": "",
    "user": "", 
    "host": "", # e.g. "sn00111.snowflakecomputing.com",
    "password": "",
    "role": "ACCOUNTADMIN",
    "warehouse": "SMALL_WH",
    "database":"MFR",
    "schema":"PUBLIC"
    }
session = Session.builder.configs(connection_parameters).create()

maintenance_df = session.table('MFR.PUBLIC.maintenance')
humidity_df = session.table('MFR.PUBLIC.Humidity')
hum_udi_df = session.table('MFR.PUBLIC.HUMIDITY_UDI')

# Look at the dataframes

In [ ]:
maintenance_df.to_pandas().head()

In [ ]:
humidity_df.to_pandas().head()

In [ ]:
hum_udi_df.to_pandas().head()

In [ ]:
# join together the dataframes and prepare training dataset
maintenance_city = maintenance_df.join(hum_udi_df, ["UDI"])
maintenance_hum = maintenance_city.join(humidity_df, (maintenance_city.col("CITY") == humidity_df.col("CITY_NAME"))).select(col("TYPE"), 
col("AIR_TEMPERATURE_K"), col("PROCESS_TEMPERATURE"), col("ROTATIONAL_SPEED_RPM"), col("TORQUE_NM"), col("TOOL_WEAR_MIN"), col("HUMIDITY_RELATIVE_AVG"), col("MACHINE_FAILURE"))

In [ ]:
# write training set to snowflake and materialize the data frame into a pandas data frame
maintenance_hum.write.mode("overwrite").save_as_table("MFR.PUBLIC.MAINTENANCE_HUM")
maintenance_hum_df = session.table('MFR.PUBLIC.MAINTENANCE_HUM').to_pandas()

In [ ]:
# drop column thats not needed
maintenance_hum_df = maintenance_hum_df.drop(columns=["TYPE"])

# Use MLFlow to track jobs and models

In [ ]:
# set up experiment
import mlflow
mlflow.set_experiment(experiment_name="predictive-maintenance")

# Split data and train model with mlflow logging

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    maintenance_hum_df.drop("MACHINE_FAILURE", axis=1), maintenance_hum_df["MACHINE_FAILURE"], test_size=0.3
)

In [ ]:
from sklearn.linear_model import LogisticRegression
mlflow.autolog()
model = LogisticRegression()


In [ ]:
run = mlflow.start_run()

In [ ]:
model.fit(X_train, y_train)

# Quick Model Metrics

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, recall_score

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

In [ ]:
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("Recall: %.2f%%" % (recall * 100.0))

In [ ]:
mlflow.end_run()

In [ ]:
run = mlflow.get_run(run.info.run_id)

# More model metrics

In [ ]:
pd.DataFrame(data=[run.data.params], index=["Value"]).T

In [ ]:
pd.DataFrame(data=[run.data.metrics], index=["Value"]).T

In [ ]:
# evaluate model on test
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay
import matplotlib.pyplot as plt
y_pred = model.predict_proba(X_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,
                                   estimator_name='example estimator')
display.plot()
plt.show()

In [ ]:
# auc score
roc_auc_score(y_test, y_pred)

In [ ]:
from sklearn.inspection import permutation_importance
feature_names = ['AIR_TEMPERATURE_K',
       'PROCESS_TEMPERATURE', 'ROTATIONAL_SPEED_RPM', 'TORQUE_NM',
       'TOOL_WEAR_MIN', 'HUMIDITY_RELATIVE_AVG']
result = permutation_importance(
    model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)

forest_importances = pd.Series(result.importances_mean, index=feature_names)
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean accuracy decrease")
fig.tight_layout()
plt.show()

# Register training dataset with mlflow runid

In [ ]:
# register dataset
from azureml.core import Workspace, Dataset

# Connect to the Workspace
ws = Workspace.from_config()

# The default datastore is a blob storage container where datasets are stored
datastore = ws.get_default_datastore()

# Register the dataset
ds = Dataset.Tabular.register_pandas_dataframe(
        dataframe=maintenance_hum_df, 
        name=run.info.run_id+'maintenance_hum_df', 
        description='maintenance df for model training',
        target=datastore
    )

In [ ]:
client = mlflow.tracking.MlflowClient()
client.list_artifacts(run_id=run.info.run_id)

# Register Model

In [ ]:
import os
mlflow.sklearn.save_model(model, "./maint")
model_local_path = os.path.abspath("./maint")
mlflow.register_model(f"file://{model_local_path}", "maint-model")

# Deploy Model to snowflake

In [ ]:
from azureml.core import Model
model_path = Model.get_model_path(model_name = 'maint', version = 1, _workspace= ws)
model_path

In [ ]:
# Load model
from joblib import load
aml_model = load(model_path + "/model.pkl")
aml_model

In [ ]:
# deploy to registry
from snowflake.ml.registry import registry
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *

reg = registry.Registry(session=session)
sample_df = maintenance_hum_df.drop("MACHINE_FAILURE", axis = 1)
reg.log_model(aml_model, model_name='aml_model', version_name='v1', sample_input_data=sample_df)

# verify model deployment and view functions associated with the model
mv = reg.get_model('aml_model').version('v1')
mv.show_functions()

In [ ]:
# run the below sql in snowflake
# -- test function
# use role accountadmin;
# select GET(aml_model!predict_proba(AIR_TEMPERATURE_K,
#       PROCESS_TEMPERATURE, ROTATIONAL_SPEED_RPM, TORQUE_NM,
#       TOOL_WEAR_MIN, HUMIDITY_RELATIVE_AVG), 'output_feature_1') as predicted_failure, * from maintenance_hum;